# Entraîner en utilisant Azure Machine Learning Compute

* Se connecter à un espace de travail Azure Machine Learning
* Utiliser une cible de calcul existante ou en créer une nouvelle
* Configurer et exécuter la commande


## Connection à l'espace de travail

Initialiser un objet d’espace de travail.

In [ ]:
# Importation des librairies
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# Les informations de notre espace de travail
subscription_id = "e91c28d"
resource_group = "for_dmr"
workspace_name = "Espace_DMR"

credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)
print(ml_client.workspace_name, workspace.resource_group, workspace.location, ml_client.connections._subscription_id, sep = '\n')

### Create project directory

Créez un répertoire qui contiendra le script de formation auquel vous devrez accéder sur la ressource distante.

In [ ]:
import os
import shutil

project_folder = os.path.join(".", 'code_folder')
os.makedirs(project_folder, exist_ok=True)
shutil.copy('script.py', project_folder)

### Use existing compute target or create new (Basic)

Azure Machine Learning Compute est une infrastructure de calcul gérée qui permet à l'utilisateur de créer facilement des calculs à nœud unique ou à nœuds multiples de la famille de machines virtuelles appropriée. Elle est créée **dans la région de votre espace de travail** et constitue une ressource qui peut être utilisée par d'autres utilisateurs de votre espace de travail. Elle se met automatiquement à l'échelle par défaut sur max_nodes, lorsqu'un travail est soumis, et s'exécute dans un environnement conteneurisé qui empaquete les dépendances comme spécifié par l'utilisateur.

Comme il s'agit d'un calcul géré, la planification des travaux et la gestion des clusters sont gérées en interne par le service Azure Machine Learning.

Un cluster de calcul peut être créé à l'aide de la classe `AmlCompute`. Certains des paramètres clés de cette classe sont :

* `size` - La taille de la machine virtuelle à utiliser pour le cluster. Pour plus d'informations, consultez [Séries et tailles de machines virtuelles prises en charge](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target#supported-vm-series-and-sizes).
* `max_instances` - Le nombre maximal de nœuds à utiliser sur le cluster. La valeur par défaut est 1.

In [ ]:
from azure.ai.ml.entities import AmlCompute

#Choisir le nom de notre cluster CPU / GPU
cluster_name = "PipelineDMR"

# On vérifie son éxistance
try:
    cluster = ml_client.compute.get(cluster_name)
    print('Found existing cluster, use it.')
except Exception:
    compute = AmlCompute(name=cluster_name, size='Standard_E4ds_v4',
                         max_instances=4)
    cluster = ml_client.compute.begin_create_or_update(compute)


### Configure & Run

L'environnement et le calcul ont été pré-remplis à partir de la tâche d'entraînement d'origine. Vous trouverez plus d'informations ici :

`command`: https://docs.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml?view=azure-python-preview#azure-ai-ml-command

`environment`: https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments#automated-ml-automl

`compute`: https://docs.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.amlcompute?view=azure-python-preview



In [ ]:
# Pour tester le script avec un environnement référencé par un fichier yaml personnalisé, supprimez le commentaire des lignes suivantes et remplacez la valeur `conda_file` par le chemin d'accès au fichier yaml.
# Définissez la valeur de `environment` dans la tâche `command` ci-dessous sur `env`.

# env = Environment(
#    name="automl-tabular-env",
#    description="environment for automl inference",
#    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20210727.v1",
#    conda_file="conda.yaml",
# )

In [ ]:
from azure.ai.ml import command, Input

# To test with new training / validation datasets, replace the default dataset id(s)/uri(s) taken from parent run below
command_str = 'python script.py --training_dataset_uri azureml://l..ocations/fling/versions/1'

command_job = command(
    code=project_folder,
    command=command_str,
    tags=dict(automl_child_run_id='dmroversampling_20'),
    environment='AzureML-ai-ml-automl:7',# en env existant
    compute='PipelineDMR',
    experiment_name='DMR-OversamplingData'
)
 
returned_job = ml_client.create_or_update(command_job)
returned_job.studio_url

### Initialize MLFlow Client

Les métriques et les artefacts de l'exécution sont accessibles via l'interface MLFlow.

Initialisez le client MLFlow ici et définissez le backend comme Azure ML, via le client MLFlow.

Cette interface est automatiquement créée lors da la création de notre espace de travail par exemple: 'Espace DMR'

*IMPORTANT*, vous devez avoir installé les derniers packages MLFlow avec :

    pip install azureml-mlflow

    pip install mlflow

In [ ]:
# %pip install azureml-mlflow
# %pip install mlflow

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Retrieve the metrics logged to the run.
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()
mlflow_run = mlflow_client.get_run(returned_job.name)
mlflow_run.data.metrics


### Download Fitted Model
Télécharger le modéle dans un répertoire local si nécessaire  `local_dir`.

In [ ]:
# import os

# Create local folder
# local_dir = "./artifact_downloads"
# if not os.path.exists(local_dir):
#     os.mkdir(local_dir)
# Download run's artifacts/outputs
# local_path = mlflow_client.download_artifacts(
#     mlflow_run.info.run_id, "outputs", local_dir
# )
# print("Artifacts downloaded in: {}".format(local_path))
# print("Artifacts: {}".format(os.listdir(local_path)))